# 📊 3D Bar Graph: Aircraft Type vs Aircraft Model vs Accident Count (First 500 Records, Titles Only)

In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
import json
import pandas as pd

# ✅ Load JSON data (first 500 records only)
file_path = r"C:\Users\olaye\Documents\UARC\recent_final_extracted_data_concise.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ Only keep the first 500 records
data = data[:500]

# Convert JSON to DataFrame
df = pd.json_normalize(data, sep='_')
df.head()

,File Name,Analysis,Probable Cause and Findings,Flight Information_Location,Flight Information_Date & Time,Flight Information_Aircraft_model,Flight Information_Aircraft,Flight Information_Defining Event,Flight Information_Flight Conducted Under,Flight Information_Accident Number,Flight Information_Registration,Flight Information_Aircraft Damage,Flight Information_Injuries
0,100004.pdf,"The pilot reported that, during landing, the a...",: The pilot's failure to maintain,"Yuba, California","June 17, 2019, 12:13 Local",Stinson 10,Stinson,Loss of control on ground,Part 91: General aviation - Personal,GAA19CA485,N34658,Substantial,Not Found
1,100246.pdf,The pilot performed a preflight inspection of ...,: A blockage of the pitot,"Bowie, Maryland","September 12, 2019, 11:20 Local",Mooney M20J,Mooney,Collision during takeoff/land,Part 91: General aviation - Personal,ERA19LA269,N202JB,Substantial,2 minor injury(ies)
2,100257.pdf,"The student pilot reported that, during landin...",: The student pilot's improper landing,"Memphis, Tennessee","September 12, 2019, 08:45 Local",Cessna 150,Cessna,Loss of control on ground,Part 91: General aviation - Instructional,GAA19CA543,N21979,Substantial,Not Found
3,100258.pdf,"The flight instructor reported that, during la...",: The flight instructor's improper decision,"Hemet, California","September 12, 2019, 15:20 Local",Cessna 172F,Cessna,Nose over/nose down,Public aircraft,GAA19CA544,N5229F,Substantial,Not Found
4,100259.pdf,"The pilot reported that, during landing, the a...",": The pilot's improper approach, which","Caldwell, Idaho","September 13, 2019, 12:45 Local",Cessna P206,Cessna,Loss of control on ground,Part 91: General aviation - Personal,GAA19CA545,N4746F,Substantial,Not Found


In [ ]:
import numpy as np

# ✅ Ensure required columns exist
if not {'Flight Information_Aircraft', 'Flight Information_Aircraft_model'}.issubset(df.columns):
    raise ValueError("❌ Required columns missing in dataset")

# ✅ Group by Aircraft and Aircraft Model
model_counts = df.groupby(['Flight Information_Aircraft', 'Flight Information_Aircraft_model']).size().unstack(fill_value=0)

aircraft_types = model_counts.index.tolist()
aircraft_models = model_counts.columns.tolist()

# ✅ Create meshgrid for bar positions
xpos, ypos = np.meshgrid(np.arange(len(aircraft_types)), np.arange(len(aircraft_models)), indexing='ij')
xpos_flat = xpos.flatten()
ypos_flat = ypos.flatten()
dz_flat = [model_counts.loc[aircraft, model] for aircraft in aircraft_types for model in aircraft_models]

In [ ]:
import plotly.graph_objects as go
import pandas as pd
fig = go.Figure()

# 🎨 Color palette for each aircraft type
color_map = {
    aircraft: color for aircraft, color in zip(
        aircraft_types,
        ['royalblue', 'firebrick', 'orange', 'purple', 'green', 'gold',
         'teal', 'deeppink', 'brown', 'darkcyan', 'navy', 'indigo'] * 10
    )
}

# ⚙️ Bar dimensions
bar_width = 0.4
bar_depth = 0.4

# 🚀 Add bars
for xi, yi, zi in zip(xpos_flat, ypos_flat, dz_flat):
    if zi == 0:
        continue
    aircraft = aircraft_types[xi]
    model = aircraft_models[yi]

    # ✍ Hover text
    hover_text = f"<b>Aircraft Type:</b> {aircraft}<br><b>Aircraft Model:</b> {model}<br><b>Number of Accidents:</b> {zi}"

    # 🔲 Cuboid corners for 3D bar
    x_corners = [xi-bar_width/2, xi-bar_width/2, xi+bar_width/2, xi+bar_width/2,
                 xi-bar_width/2, xi-bar_width/2, xi+bar_width/2, xi+bar_width/2]
    y_corners = [yi-bar_depth/2, yi+bar_depth/2, yi+bar_depth/2, yi-bar_depth/2,
                 yi-bar_depth/2, yi+bar_depth/2, yi+bar_depth/2, yi-bar_depth/2]
    z_corners = [0, 0, 0, 0, zi, zi, zi, zi]

    # 🎨 Add colored 3D bar
    fig.add_trace(go.Mesh3d(
        x=x_corners,
        y=y_corners,
        z=z_corners,
        color=color_map[aircraft],
        opacity=0.85,
        i=[0, 0, 0, 1, 2, 4, 5, 6, 4, 5, 1, 2],
        j=[1, 2, 3, 5, 6, 5, 6, 7, 0, 1, 5, 6],
        k=[5, 6, 7, 6, 7, 1, 2, 3, 5, 6, 0, 1],
        name=f"{aircraft}",
        hovertemplate=hover_text
    ))

# 🎛 Layout: only show titles, hide tick labels
fig.update_layout(
    title=dict(
        text='3D Bar Graph: Aircraft Type vs Aircraft Model vs Number of Accidents (First 500 Records)',
        font=dict(size=14)
    ),
    width=1400,
    height=850,
    scene=dict(
        xaxis=dict(
            title=dict(text='Aircraft Type', font=dict(size=12, color='black')),
            tickvals=[],
            ticktext=[],
            showticklabels=False,
            showline=True,
            linecolor='black',
            linewidth=3
        ),
        yaxis=dict(
            title=dict(text='Aircraft Model', font=dict(size=12, color='black')),
            tickvals=[],
            ticktext=[],
            showticklabels=False,
            showline=True,
            linecolor='black',
            linewidth=3
        ),
        zaxis=dict(
            title=dict(text='Accident Count', font=dict(size=12, color='black')),
            tickvals=[],
            ticktext=[],
            showticklabels=False,
            showline=True,
            linecolor='black',
            linewidth=3
        )
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    template='plotly_white',
    showlegend=False
)

fig.show()

In [ ]:
import json
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.io as pio

# --- Choose best renderer automatically ---
def set_renderer():
    try:
        # Try normal notebook renderer
        pio.renderers.default = "notebook"
        return "notebook"
    except Exception:
        try:
            # If that fails, try notebook_connected
            pio.renderers.default = "notebook_connected"
            return "notebook_connected"
        except Exception:
            # Final fallback: browser
            pio.renderers.default = "browser"
            return "browser"

renderer_used = set_renderer()
print(f"✅ Plotly renderer set to: {renderer_used}")

# --- Load dataset ---
file_path = r"C:\Users\olaye\Documents\UARC\final_extracted_data1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Extract dates and aircraft ---
records = []
for item in data:
    fi = item.get("Flight Information", {})
    date_str = fi.get("Date & Time")
    aircraft = fi.get("Aircraft")
    
    if date_str and aircraft:
        try:
            # Parse date (strip time, keep only Y-M-D)
            date_part = date_str.split(",")[0].strip() + " " + date_str.split(",")[1].strip().split()[0]
            parsed_date = datetime.strptime(date_part, "%B %d %Y")
            date_clean = parsed_date.strftime("%Y-%m-%d")
            records.append({"date": date_clean, "aircraft": aircraft})
        except Exception:
            continue

# --- Convert to DataFrame ---
df = pd.DataFrame(records)

if df.empty:
    print("⚠️ No records extracted. Check JSON structure.")
else:
    grouped = df.groupby("date")["aircraft"].apply(list).reset_index()
    grouped["count"] = grouped["aircraft"].apply(len)

    # --- Create 3D interactive chart ---
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=list(range(len(grouped))),      # Date index
        y=grouped["count"],              # Aircraft count
        z=[0]*len(grouped),              # Flat z-axis
        mode="markers",
        marker=dict(
            size=8,
            color=grouped["count"],
            colorscale="Viridis",
            opacity=0.8
        ),
        text=[f"{d}<br>Aircraft: {', '.join(a[:5])}{' ...' if len(a)>5 else ''}" 
              for d, a in zip(grouped["date"], grouped["aircraft"])],
        hoverinfo="text"
    ))

    fig.update_layout(
        title="3D Visualization of Aircraft by Date",
        scene=dict(
            xaxis=dict(title="Date Index"),
            yaxis=dict(title="Number of Aircraft"),
            zaxis=dict(title="Z-axis (placeholder)")
        ),
        margin=dict(l=0, r=0, t=50, b=0)
    )

    pio.renderers.default = "browser"

    # --- Show preview table ---
    display(grouped.head(10))


✅ Plotly renderer set to: notebook


,date,aircraft,count
0,2016-09-10,"[Beech, HEFTY, Luscombe, DIETERICH]",4
1,2016-09-11,"[Piper, Piper, Cessna, Cessna, Mooney]",5
2,2016-09-13,[Piper],1
3,2016-09-14,"[Stinson, Piper, Piper]",3
4,2016-09-15,[Piper],1
5,2016-09-16,"[Cessna, Cessna, Helio, JEFFERY, Beech]",5
6,2017-02-24,"[Cessna, Cessna, Cessna]",3
7,2017-02-25,"[Piper, Cessna, Cessna, Socata]",4
8,2017-02-26,[Ryan],1
9,2017-02-27,"[Cessna, Cessna, Socata, Piper]",4


In [ ]:
import json

file_path = r"C:\Users\olaye\Documents\UARC\final_extracted_data1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Print first item to inspect structure
print(json.dumps(data[0], indent=2))


In [ ]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# --- Safe renderer selection ---
def safe_renderer():
    """Try inline rendering, fallback to browser if nbformat/ipython not available."""
    try:
        pio.renderers.default = "notebook_connected"
        return "notebook_connected"
    except Exception:
        try:
            pio.renderers.default = "notebook"
            return "notebook"
        except Exception:
            pio.renderers.default = "browser"
            return "browser"

renderer_used = safe_renderer()
print(f"✅ Using Plotly renderer: {renderer_used}")

# --- Load dataset ---
file_path = r"C:\Users\olaye\Documents\UARC\final_extracted_data1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Extract records (keep original date style) ---
records = []
for item in data:
    fi = item.get("Flight Information", {})
    date_str = fi.get("Date & Time")
    aircraft = fi.get("Aircraft")
    model = fi.get("Aircraft_model")

    if date_str and aircraft:
        # Extract only "Month Day, Year"
        parts = date_str.split(",")
        if len(parts) >= 2:
            date_clean = parts[0].strip() + ", " + parts[1].strip().split()[0]
        else:
            date_clean = date_str  # fallback if unusual format
        records.append({
            "date": date_clean,
            "aircraft": aircraft,
            "aircraft_model": model if model else "Unknown"
        })

# --- Convert to DataFrame ---
df = pd.DataFrame(records)

if df.empty:
    print("⚠️ No records extracted. Please check JSON structure again.")
else:
    # --- Group by date ---
    grouped = df.groupby("date").agg({
        "aircraft": list,
        "aircraft_model": list
    }).reset_index()
    grouped["count"] = grouped["aircraft"].apply(len)

    # --- Main 3D Graph ---
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=list(range(len(grouped))),
        y=grouped["count"],
        z=[0]*len(grouped),
        mode="markers",
        marker=dict(
            size=8,
            color=grouped["count"],
            colorscale="Viridis",
            opacity=0.8
        ),
        text=[f"<b>{d}</b><br>Aircraft: {', '.join(a[:5])}{' ...' if len(a) > 5 else ''}" 
              for d, a in zip(grouped["date"], grouped["aircraft"])],
        hoverinfo="text"
    ))

    fig.update_layout(
        title="3D Visualization of Aircraft by Date",
        scene=dict(
            xaxis=dict(title="Date Index"),
            yaxis=dict(title="Aircraft Count"),
            zaxis=dict(title="Z-axis (placeholder)")
        ),
        margin=dict(l=0, r=0, t=50, b=0)
    )

    pio.renderers.default = "browser" 

    fig.show()



✅ Using Plotly renderer: notebook_connected


'\'\'\n    # --- Search Feature ---\n    def show_date_graph(search_date):\n        """Show a graph for a specific date in original format."""\n        subset = df[df["date"] == search_date]\n\n        if subset.empty:\n            print(f"⚠️ No records found for {search_date}")\n            return\n\n        fig = go.Figure()\n        fig.add_trace(go.Bar(\n            x=subset["aircraft_model"],\n            y=[1]*len(subset),\n            text=subset["aircraft"],\n            hoverinfo="text",\n            marker=dict(color="rgba(50, 150, 250, 0.7)")\n        ))\n\n        fig.update_layout(\n            title=f"Aircraft on {search_date}",\n            xaxis=dict(title="Aircraft Model"),\n            yaxis=dict(title="Count"),\n            margin=dict(l=40, r=40, t=60, b=40)\n        )\n\n        pio.renderers.default = "browser" \n\n        fig.show()\n\n    # Example search\n    show_date_graph("June 17, 2019")\n'

In [ ]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# --- Always open in browser ---
pio.renderers.default = "browser"

# --- Load dataset ---
file_path = r"C:\Users\olaye\Documents\UARC\final_extracted_data1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Extract records ---
records = []
for item in data:
    fi = item.get("Flight Information", {})
    date_str = fi.get("Date & Time")
    aircraft = fi.get("Aircraft")
    model = fi.get("Aircraft_model")

    if date_str and aircraft:
        # Keep original "Month Day, Year"
        parts = date_str.split(",")
        if len(parts) >= 2:
            date_clean = parts[0].strip() + ", " + parts[1].strip().split()[0]
        else:
            date_clean = date_str
        records.append({
            "date": date_clean,
            "aircraft": aircraft,
            "aircraft_model": model if model else "Unknown"
        })

df = pd.DataFrame(records)

# --- Group by date ---
grouped = df.groupby("date").agg({
    "aircraft": list,
    "aircraft_model": list
}).reset_index()
grouped["count"] = grouped["aircraft"].apply(len)

# --- Manufacturer → Color mapping ---
manufacturer_colors = {
    "Cessna": "orange",
    "Piper": "green",
    "Boeing": "blue",
    "Mooney": "purple",
    "Beech": "red",
    "Airbus": "teal",
    "Douglas": "brown",
    "Stinson": "pink",
    "Unknown": "gray"
}

def get_color_for_model(model):
    for manu in manufacturer_colors.keys():
        if manu.lower() in model.lower():
            return manufacturer_colors[manu]
    return "gray"  # fallback

# --- Main 3D Figure ---
fig = go.Figure()

# Add "All Dates" 3D scatter
fig.add_trace(go.Scatter3d(
    x=list(range(len(grouped))),
    y=grouped["count"],
    z=[0]*len(grouped),
    mode="markers",
    marker=dict(size=8, color=grouped["count"], colorscale="Viridis", opacity=0.8),
    text=[f"<b>{d}</b><br>Aircraft: {', '.join(a[:5])}{' ...' if len(a)>5 else ''}" 
          for d, a in zip(grouped["date"], grouped["aircraft"])],
    hoverinfo="text",
    name="All Dates"
))

# Add per-date bar charts (hidden initially)
for d in grouped["date"]:
    subset = df[df["date"] == d]
    colors = [get_color_for_model(m) for m in subset["aircraft_model"]]

    fig.add_trace(go.Bar(
        x=subset["aircraft_model"],
        y=[1]*len(subset),
        text=subset["aircraft"],
        hoverinfo="text",
        marker=dict(color=colors),
        name=d,
        visible=False
    ))

# --- Dropdown menu ---
buttons = []
buttons.append(dict(
    label="All Dates (3D View)",
    method="update",
    args=[{"visible": [True] + [False]*len(grouped)},
          {"title": "3D Visualization of Aircraft by Date"}]
))

for i, d in enumerate(grouped["date"], start=1):
    vis = [False]*(len(grouped)+1)
    vis[i] = True
    buttons.append(dict(
        label=d,
        method="update",
        args=[{"visible": vis},
              {"title": f"Aircraft on {d} (Color-coded by Manufacturer)"}]
    ))

fig.update_layout(
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        x=1.2, y=1.05
    )],
    title="3D Visualization of Aircraft by Date",
    scene=dict(
        xaxis=dict(title="Date Index"),
        yaxis=dict(title="Aircraft Count"),
        zaxis=dict(title="Z-axis")
    ),
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()


In [ ]:


" This is for displaying different dates per graph"

import json
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

# --- Load your JSON file ---
with open("C:/Users/olaye/Documents/UARC/final_extracted_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Extract and clean data ---
records = []
for entry in data:
    info = entry.get("Flight Information", {})
    date = info.get("Date & Time", "").split(",")[0].strip()
    aircraft = info.get("Aircraft", "Unknown")
    model = info.get("Aircraft_model", "Unknown")
    
    if date and aircraft:
        records.append({
            "date": date,
            "aircraft": aircraft,
            "aircraft_model": model
        })

df = pd.DataFrame(records)
print("✅ Preview of extracted data:")
display(df.head(10))

# --- Group by date ---
grouped = df.groupby("date").agg({
    "aircraft": list,
    "aircraft_model": list
}).reset_index()

grouped["count"] = grouped["aircraft"].apply(len)

# --- Create 3D visualization of all dates ---
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=list(range(len(grouped))),            # index for spacing
    y=grouped["count"],                     # PDF count per date
    z=[0] * len(grouped),                   # flat z-axis for layout
    mode="markers+text",
    marker=dict(size=6, color='blue'),
    text=grouped["date"],
    hovertext=[
        f"<b>Date:</b> {row['date']}<br><b>Aircraft:</b> {', '.join(row['aircraft'][:5])}"
        + ("<br>...and more" if len(row['aircraft']) > 5 else "")
        for _, row in grouped.iterrows()
    ],
    hoverinfo="text"
))

fig.update_layout(
    title="🛩️ 3D Visualization of Aircraft by Date",
    scene=dict(
        xaxis_title="Date Index",
        yaxis_title="Aircraft Count",
        zaxis_title="Flat Axis (Z=0)"
    ),
    margin=dict(l=0, r=0, t=40, b=0)
)

# Display the full graph of all dates
fig.show()

# --- Keyword Search with input() ---
search_date = input("\n🔍 Enter a date to search (e.g. June 17, 2019): ").strip()

matching_rows = df[df["date"] == search_date]

if matching_rows.empty:
    print(f"❌ No results found for: {search_date}")
else:
    print(f"\n✅ Results for {search_date}:")
    display(matching_rows)

    # Optional mini 3D graph for selected date
    fig2 = go.Figure()

    fig2.add_trace(go.Scatter3d(
        x=list(range(len(matching_rows))),
        y=[1] * len(matching_rows),
        z=[0] * len(matching_rows),
        mode="markers+text",
        marker=dict(size=6, color='green'),
        text=matching_rows["aircraft_model"],
        hovertext=[
            f"<b>Aircraft:</b> {row['aircraft']}<br><b>Model:</b> {row['aircraft_model']}"
            for _, row in matching_rows.iterrows()
        ],
        hoverinfo="text"
    ))

    fig2.update_layout(
        title=f"🛬 Aircraft Incidents on {search_date}",
        scene=dict(
            xaxis_title="Incident Index",
            yaxis_title="Occurrence (Fixed)",
            zaxis_title="Z = 0"
        ),
        margin=dict(l=0, r=0, t=50, b=0)
    )

    fig2.show()


✅ Preview of extracted data:


,date,aircraft,aircraft_model
0,June 17,Stinson,Stinson 10
1,September 12,Mooney,Mooney M20J
2,September 12,Cessna,Cessna 150
3,September 12,Cessna,Cessna 172F
4,September 13,Cessna,Cessna P206
5,September 14,Mooney,Mooney M20M
6,September 13,Piper,Piper PA28
7,September 13,Cirrus,Cirrus SR22
8,September 15,Piper,Piper PA46
9,September 16,Airbus,Airbus A220



✅ Results for September 19:


,date,aircraft,aircraft_model
505,September 19,Vans,Vans RV9
506,September 19,Vans,Vans RV4
509,September 19,North,North American B25N
515,September 19,Air,Air Tractor AT 402
534,September 19,Piper,Piper PA31
619,September 19,Cessna,Cessna 210
675,September 19,Bombardier,Bombardier CL600 2C10
1691,September 19,Piper,Piper PA-22-150
1693,September 19,Cessna,Cessna 172N
1705,September 19,Piper,Piper PA28


In [12]:
import json
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

# --- Load the dataset ---
with open("C:/Users/olaye/Documents/UARC/final_extracted_data1.json", "r", encoding="utf-8") as f:
    records = json.load(f)

# --- Extract and normalize ---
flattened = []
for record in records:
    flight_info = record.get("Flight Information", {})
    raw_date = flight_info.get("Date & Time", "")  # Example: "July 27, 2019, 16:17 Local"
    
    try:
        date_parts = raw_date.split(",")  # ['July 27', ' 2019', ' 16:17 Local']
        if len(date_parts) >= 2:
            date = f"{date_parts[0].strip()}, {date_parts[1].strip()}"  # e.g. "July 27, 2019"
        else:
            date = raw_date.strip()
    except:
        date = "Unknown"

    aircraft = flight_info.get("Aircraft", "Unknown")
    aircraft_model = flight_info.get("Aircraft_model", "Unknown")

    if date != "Unknown":
        flattened.append({
            "date": date,
            "aircraft": aircraft,
            "aircraft_model": aircraft_model
        })

df = pd.DataFrame(flattened)

# --- Group by date ---
grouped = df.groupby("date").agg({
    "aircraft": list,
    "aircraft_model": list
}).reset_index()
grouped["count"] = grouped["aircraft"].apply(len)

# --- 3D Graph of All Dates ---
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=list(range(len(grouped))),
    y=grouped["count"],
    z=[0]*len(grouped),
    mode="markers+text",
    text=grouped["date"],
    marker=dict(size=8, color=grouped["count"], colorscale="Viridis", opacity=0.8),
    hoverinfo="text"
))
fig.update_layout(
    title="📅 3D Visualization of Aircraft Incidents by Date",
    scene=dict(
        xaxis_title="Date Index",
        yaxis_title="Aircraft Count",
        zaxis_title="",
        camera=dict(eye=dict(x=1.5, y=1.5, z=0.3))
    ),
    margin=dict(l=0, r=0, t=50, b=0)
)
fig.show()

# --- Normalize for Searching ---
grouped["clean_date"] = grouped["date"].str.replace(",", "").str.replace("  ", " ").str.strip().str.lower()

# --- Search Prompt for Specific Date ---
search_date = input("Enter a date (e.g. September 12, 2019): ").strip()
search_date_clean = search_date.replace(",", "").replace("  ", " ").strip().lower()

if search_date_clean in grouped["clean_date"].values:
    result = grouped[grouped["clean_date"] == search_date_clean].iloc[0]
    aircraft = result["aircraft"]
    models = result["aircraft_model"]

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter3d(
        x=list(range(len(aircraft))),
        y=[1]*len(aircraft),
        z=[0]*len(aircraft),
        mode="markers+text",
        text=[f"{a} ({m})" for a, m in zip(aircraft, models)],
        marker=dict(size=10, color='orange'),
        hoverinfo="text"
    ))
    fig2.update_layout(
        title=f"✈️ Aircraft on {search_date}",
        scene=dict(
            xaxis_title="Aircraft Index",
            yaxis_title="Constant",
            zaxis_title=""
        ),
        margin=dict(l=0, r=0, t=50, b=0)
    )
    fig2.show()
else:
    print(f"❌ No data found for '{search_date}'. Please check the format or try another date.")


In [2]:
import json
import pandas as pd
from datetime import datetime
import plotly.graph_objs as go

# Load the dataset (your exact path)
file_path = "C:/Users/olaye/Documents/UARC/final_extracted_data1.json"
with open(file_path, "r", encoding="utf-8") as f:
    records = json.load(f)

# Extract fields with full date
flattened = []
for record in records:
    flight_info = record.get("Flight Information", {})
    date_time = flight_info.get("Date & Time", "").strip()
    aircraft = flight_info.get("Aircraft", "Unknown")
    aircraft_model = flight_info.get("Aircraft_model", "Unknown")

    try:
        date_part = date_time.split(",")[0].strip() + ", " + date_time.split(",")[1].strip().split()[0]
        parsed_date = datetime.strptime(date_part, "%B %d, %Y")
        formatted_date = parsed_date.strftime("%B %d, %Y")
    except (IndexError, ValueError):
        continue

    flattened.append({
        "date": formatted_date,
        "aircraft": aircraft,
        "aircraft_model": aircraft_model
    })

# Create DataFrame
df = pd.DataFrame(flattened)

# Group by date
grouped = df.groupby("date").agg({
    "aircraft": list,
    "aircraft_model": list
}).reset_index()

# Create density values
grouped["incident_count"] = grouped["aircraft"].apply(len)
grouped["x"] = range(len(grouped))  # Position on x-axis
grouped["y"] = grouped["incident_count"]  # Height
grouped["z"] = [0]*len(grouped)  # Base

# Hover text for details
grouped["hover"] = grouped.apply(lambda row: "<br>".join([
    f"📅 {row['date']}",
    f"✈️ Incidents: {row['incident_count']}",
    *[f"{a} ({m})" for a, m in zip(row['aircraft'], row['aircraft_model'])]
]), axis=1)

# 3D Scatter plot simulating "document towers"
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=grouped["x"],
    y=grouped["y"],
    z=grouped["z"],
    mode="markers",
    marker=dict(
        size=grouped["incident_count"],
        color=grouped["incident_count"],
        colorscale="Purples",
        opacity=0.9,
        colorbar=dict(title="Incident Count")
    ),
    hovertext=grouped["hover"],
    hoverinfo="text"
))

fig.update_layout(
    title="🟣 FAA Incident Document Tower (Darker = More Reports)",
    scene=dict(
        xaxis_title="Date Index",
        yaxis_title="Incident Count",
        zaxis_title="",
        camera=dict(eye=dict(x=1.3, y=1.3, z=0.6))
    ),
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()


In [1]:
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dash
from dash import html, dcc, Input, Output, State
from datetime import datetime

# --- Load and process the JSON dataset ---
with open("C:/Users/olaye/Documents/UARC/final_extracted_data1.json", "r", encoding="utf-8") as f:
    records = json.load(f)

flattened = []
for record in records:
    flight_info = record.get("Flight Information", {})
    full_date = flight_info.get("Date & Time", "").split(",")[0].strip()
    aircraft = flight_info.get("Aircraft", "Unknown")
    model = flight_info.get("Aircraft_model", "Unknown")
    
    if full_date:
        flattened.append({
            "date": full_date,
            "aircraft": aircraft,
            "model": model
        })

df = pd.DataFrame(flattened)

# --- Group by date ---
grouped = df.groupby("date").agg({
    "aircraft": list,
    "model": list,
    "date": "count"
}).rename(columns={"date": "incident_count"}).reset_index()

# Create 3D grid layout (like a city)
n = len(grouped)
grid_size = int(np.ceil(np.sqrt(n)))

# Assign grid positions
grouped["grid_x"] = grouped.index % grid_size
grouped["grid_y"] = grouped.index // grid_size
grouped["height"] = grouped["incident_count"]

# Normalize height for visualization
max_height = grouped["height"].max()
grouped["norm_height"] = grouped["height"] / max_height * 50

# Create volumetric boxes/towers
def create_volumetric_towers(data):
    """Create multiple stacked cubes to simulate volumetric density"""
    x_all, y_all, z_all = [], [], []
    colors_all = []
    hover_all = []
    
    for _, row in data.iterrows():
        height_levels = int(row["norm_height"])
        if height_levels < 1:
            height_levels = 1
            
        # Create stacked boxes
        for level in range(height_levels):
            # Box corners
            x_base = row["grid_x"]
            y_base = row["grid_y"]
            z_base = level * 2
            
            # Create box vertices
            box_x = [x_base, x_base+1, x_base+1, x_base, x_base,
                     x_base, x_base+1, x_base+1, x_base, x_base]
            box_y = [y_base, y_base, y_base+1, y_base+1, y_base,
                     y_base, y_base, y_base+1, y_base+1, y_base]
            box_z = [z_base, z_base, z_base, z_base, z_base,
                     z_base+2, z_base+2, z_base+2, z_base+2, z_base+2]
            
            x_all.extend(box_x)
            y_all.extend(box_y)
            z_all.extend(box_z)
            
            # Color based on density (darker = more incidents)
            opacity = min(0.3 + (level / height_levels) * 0.7, 1.0)
            colors_all.extend([row["incident_count"]] * len(box_x))
            hover_all.extend([f"{row['date']}<br>Incidents: {row['incident_count']}<br>Level: {level+1}"] * len(box_x))
    
    return x_all, y_all, z_all, colors_all, hover_all

# --- Dash App ---
app = dash.Dash(__name__)
app.title = "3D Volumetric Aircraft Incident Visualizer"

app.layout = html.Div([
    html.H1("📊 3D Volumetric Aircraft Incident Visualization", 
            style={'textAlign': 'center', 'color': '#333', 'marginBottom': '10px'}),
    html.P("Dense (darker) areas represent higher incident density. Click or search dates to explore details.",
           style={'textAlign': 'center', 'color': '#666', 'marginBottom': '20px'}),
    
    html.Div([
        dcc.Input(
            id='date-input',
            type='text',
            placeholder='Search date (e.g., September 12, 2019)',
            style={'width': '70%', 'padding': '12px', 'fontSize': '14px', 
                   'border': '2px solid #ddd', 'borderRadius': '4px'}
        ),
        html.Button('Search', id='search-btn', n_clicks=0,
                   style={'padding': '12px 24px', 'marginLeft': '10px', 
                          'backgroundColor': '#7B68EE', 'color': 'white',
                          'border': 'none', 'borderRadius': '4px', 'cursor': 'pointer'})
    ], style={'textAlign': 'center', 'marginBottom': '20px'}),
    
    dcc.Graph(id='3d-volumetric-graph', style={'height': '700px'}),
    
    html.Div(id='details-output', style={
        'marginTop': '20px', 
        'padding': '20px', 
        'backgroundColor': '#f9f9f9',
        'borderRadius': '8px',
        'maxWidth': '800px',
        'margin': '20px auto'
    })
])

@app.callback(
    Output('3d-volumetric-graph', 'figure'),
    Output('details-output', 'children'),
    Input('search-btn', 'n_clicks'),
    Input('3d-volumetric-graph', 'clickData'),
    State('date-input', 'value')
)
def update_visualization(n_clicks, click_data, date_query):
    selected_date = None
    
    # Handle search input
    if date_query:
        filtered = grouped[grouped["date"].str.lower() == date_query.strip().lower()]
        if not filtered.empty:
            selected_date = filtered.iloc[0]["date"]
    
    # Handle click on graph
    if click_data and 'points' in click_data:
        hover_text = click_data['points'][0].get('text', '')
        if '<br>' in hover_text:
            selected_date = hover_text.split('<br>')[0]
    
    # Create the volumetric visualization
    fig = go.Figure()
    
    # Generate towers with volumetric effect
    for _, row in grouped.iterrows():
        height = int(row["norm_height"])
        if height < 1:
            height = 1
        
        # Create multiple mesh cubes for each tower
        for level in range(0, height, 2):
            x_base, y_base = row["grid_x"], row["grid_y"]
            size = 0.9  # Box size
            
            # Cube vertices
            vertices_x = [x_base, x_base+size, x_base+size, x_base, 
                         x_base, x_base+size, x_base+size, x_base]
            vertices_y = [y_base, y_base, y_base+size, y_base+size,
                         y_base, y_base, y_base+size, y_base+size]
            vertices_z = [level, level, level, level,
                         level+2, level+2, level+2, level+2]
            
            # Calculate opacity (denser at higher levels)
            opacity = 0.2 + (level / height) * 0.6
            
            # Highlight selected tower
            color = 'red' if selected_date == row["date"] else 'rgb(123, 104, 238)'
            
            fig.add_trace(go.Mesh3d(
                x=vertices_x,
                y=vertices_y,
                z=vertices_z,
                i=[0, 0, 0, 0, 4, 4, 6, 1],
                j=[1, 2, 3, 4, 5, 6, 5, 2],
                k=[2, 3, 4, 5, 6, 7, 2, 6],
                opacity=opacity,
                color=color,
                hovertext=f"{row['date']}<br>Incidents: {row['incident_count']}",
                hoverinfo='text',
                showlegend=False
            ))
    
    # Update layout
    fig.update_layout(
        title=dict(
            text="FAA Incident Data - Volumetric Density Visualization",
            font=dict(size=18, color='#333')
        ),
        scene=dict(
            xaxis=dict(title='', showgrid=True, gridcolor='lightgray', showticklabels=False),
            yaxis=dict(title='', showgrid=True, gridcolor='lightgray', showticklabels=False),
            zaxis=dict(title='Incident Density', showgrid=True, gridcolor='lightgray'),
            bgcolor='rgb(240, 240, 245)',
            camera=dict(
                eye=dict(x=1.8, y=1.8, z=0.8),
                center=dict(x=0, y=0, z=0)
            )
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        paper_bgcolor='white',
        showlegend=False
    )
    
    # Details panel
    if selected_date:
        idx = df["date"] == selected_date
        rows = df[idx][["aircraft", "model"]]
        
        details = html.Div([
            html.H3(f"✈️ Incident Details: {selected_date}", 
                   style={'color': '#7B68EE', 'marginBottom': '15px'}),
            html.P(f"Total Incidents: {len(rows)}", 
                  style={'fontSize': '16px', 'fontWeight': 'bold'}),
            html.Hr(),
            html.Ul([
                html.Li(f"{row.aircraft} - {row.model}", 
                       style={'padding': '5px 0'}) 
                for row in rows.itertuples()
            ], style={'listStyle': 'none', 'padding': '0'})
        ])
        return fig, details
    elif date_query:
        return fig, html.Div(f"❌ No incidents found for '{date_query}'", 
                            style={'color': '#d32f2f', 'textAlign': 'center'})
    else:
        return fig, html.Div("💡 Click on any tower or search for a date to view incident details", 
                            style={'color': '#666', 'textAlign': 'center', 'fontStyle': 'italic'})

# --- Run ---
if __name__ == "__main__":
    app.run_server(debug=True)